## Image Classifier Using VGG16 model and testing performance with ResNet50 model

Use the <code>wget</code> command to download the data from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2021-05-08 15:08:18--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  36.7MB/s    in 7.5s    

2021-05-08 15:08:26 (33.3 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [2]:
!unzip -q concrete_data_week4.zip

In [23]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [24]:
pwd

'/content'

In [25]:
num_classes = 2
batch_size = 100
image_size = (224, 224)

In [38]:
generator = ImageDataGenerator(preprocessing_function=preprocess_input)

training_generator = generator.flow_from_directory(
    "concrete_data_week4/train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
)
validation_generator = generator.flow_from_directory(
    "concrete_data_week4/valid",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [39]:
model_vgg16 = Sequential()

model_vgg16.add(VGG16(include_top=False, pooling="avg", weights="imagenet",))
model_vgg16.add(Dense(num_classes, activation="softmax"))

model_vgg16.layers[0].trainable = False

model_vgg16.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [40]:
model_vgg16.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [41]:
num_epochs = 2
steps_per_epoch_training = len(training_generator)
steps_per_epoch_validation = len(validation_generator)

history_vgg16 = model_vgg16.fit_generator(
    training_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
301/301 [==============================] - 300s 995ms/step - loss: 0.2189 - accuracy: 0.9020 - val_loss: 0.0244 - val_accuracy: 0.9955
Epoch 2/2
301/301 [==============================] - 299s 995ms/step - loss: 0.0210 - accuracy: 0.9958 - val_loss: 0.0142 - val_accuracy: 0.9968


In [42]:
model_vgg16.save("/content")

INFO:tensorflow:Assets written to: /content/assets


In [43]:
from keras.models import load_model

model_resnet50 = load_model("/content")

In [46]:
testing_generator = generator.flow_from_directory(
    "concrete_data_week4/test", target_size=image_size, shuffle=False,
)

Found 500 images belonging to 2 classes.


In [47]:
performance_vgg16 = model_vgg16.evaluate_generator(testing_generator)
print("Performance of the VGG16-trained model")
print("Loss: {}".format(round(performance_vgg16[0], 5)))
print("Accuracy: {}".format(round(performance_vgg16[1], 5)))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Performance of the VGG16-trained model
Loss: 0.01258
Accuracy: 0.998


In [48]:
performance_resnet50 = model_resnet50.evaluate_generator(testing_generator)
print("Performance of the ResNet50-trained model")
print("Loss: {}".format(round(performance_resnet50[0], 5)))
print("Accuracy: {}".format(round(performance_resnet50[1], 5)))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Performance of the ResNet50-trained model
Loss: 0.01258
Accuracy: 0.998


In this model, we will predict whether the images in the test data are images of cracked concrete or not.

In [44]:
predictions_vgg16 = model_vgg16.predict_generator(testing_generator, steps=1)


def print_prediction(prediction):
    if prediction[0] > prediction[1]:
        print("Negative ({}% certainty)".format(round(prediction[0] * 100, 1)))
    elif prediction[1] > prediction[0]:
        print("Positive ({}% certainty)".format(round(prediction[1] * 100, 1)))
    else:
        print("Unsure (prediction split 50–50)")


print("First five predictions for the VGG16-trained model:")
for i in range(5):
    print_prediction(predictions_vgg16[i])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


First five predictions for the VGG16-trained model:
Negative (99.8% certainty)
Negative (94.4% certainty)
Negative (95.8% certainty)
Negative (99.2% certainty)
Negative (99.0% certainty)


In [45]:
predictions_resnet50 = model_resnet50.predict_generator(testing_generator, steps=1)
print("First five predictions for the ResNet50-trained model:")
for i in range(5):
    print_prediction(predictions_resnet50[i])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


First five predictions for the ResNet50-trained model:
Negative (99.8% certainty)
Negative (94.4% certainty)
Negative (95.8% certainty)
Negative (99.2% certainty)
Negative (99.0% certainty)
